<a href="https://colab.research.google.com/github/tejassui/Sentiment-Analysis-DistilBERT/blob/main/fine_tuning_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
!pip install transformers[torch]
!pip install datasets transformers==4.30.0
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 96.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0


In [ ]:
!pip install datasets transformers huggingface_hub

In [ ]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [ ]:
from datasets import load_dataset
emot = load_dataset("dair-ai/emotion")

In [ ]:
small_train_dataset = emot["train"].shuffle(seed=42).select([i for i in list(range(16000))])
small_test_dataset = emot["test"].shuffle(seed=42).select([i for i in list(range(2000))])

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
  return tokenizer(examples["text"], truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched = True)
tokenized_test = small_test_dataset.map(preprocess_function, batched = True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=6)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier.bias', 'pre_classifier.we

In [ ]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
  load_accuracy = load_metric("accuracy")
  load_f1 = load_metric("f1")

  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
  f1 = load_f1.compute(predictions=predictions, references=labels, average = "weighted")["f1"]
  return {"accuracy": accuracy, "f1": f1}

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

('4.31.0', '0.21.0')

In [ ]:
pip install accelerate -U

In [ ]:
from transformers import TrainingArguments, Trainer

repo_name = "finetuning-emotion-model-16000-samples"

training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/tkathuria/finetuning-emotion-model-16000-samples into local empty directory.


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.704400
1000,0.256000
1500,0.151200
2000,0.147600


TrainOutput(global_step=2000, training_loss=0.31479635238647463, metrics={'train_runtime': 200.0914, 'train_samples_per_second': 159.927, 'train_steps_per_second': 9.995, 'total_flos': 389125901966976.0, 'train_loss': 0.31479635238647463, 'epoch': 2.0})

In [ ]:
! pip install evaluate

In [ ]:
trainer.evaluate()

<ipython-input-8-14d24b25b92e>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


{'eval_loss': 0.17905071377754211,
 'eval_accuracy': 0.921,
 'eval_f1': 0.9213743527314219,
 'eval_runtime': 5.9397,
 'eval_samples_per_second': 336.717,
 'eval_steps_per_second': 21.045,
 'epoch': 2.0}

In [ ]:
trainer.push_to_hub()

Upload file runs/Aug08_02-34-25_887938ccfc30/events.out.tfevents.1691462095.887938ccfc30.6820.0: 100%|########…

To https://huggingface.co/tkathuria/finetuning-emotion-model-16000-samples
   80fc230..89910ad  main -> main

   80fc230..89910ad  main -> main

To https://huggingface.co/tkathuria/finetuning-emotion-model-16000-samples
   89910ad..01eaa94  main -> main

   89910ad..01eaa94  main -> main



'https://huggingface.co/tkathuria/finetuning-emotion-model-16000-samples/commit/89910ad3127b8a93bb4f5633969a92d7f115aff6'

In [ ]:
! pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 10.0 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline

sentiment_model = pipeline(model="tkathuria/finetuning-emotion-model-3000-samples")
sentiment_model(["I love this movie this shit bangs", "This movie fucking sucks ughhh i hate it!"])

[{'label': 'LABEL_1', 'score': 0.5899664163589478},
 {'label': 'LABEL_3', 'score': 0.565131425857544}]

In [ ]:
from transformers import pipeline

sentiment_model = pipeline(model="tkathuria/finetuning-emotion-model-12000-samples")
sentiment_model(["I LOVE this movie this movie is a hit", "This movie fucking sucks ughhh i hate it!"])

[{'label': 'LABEL_1', 'score': 0.9788538217544556},
 {'label': 'LABEL_3', 'score': 0.9978983402252197}]

In [ ]:
from transformers import pipeline

sentiment_model = pipeline(model="tkathuria/finetuning-emotion-model-16000-samples")
sentiment_model(["I LOVE this movie this movie is a hit", "This movie fucking sucks ughhh i hate it!"])

[{'label': 'LABEL_1', 'score': 0.940116822719574},
 {'label': 'LABEL_3', 'score': 0.9907101988792419}]

In [ ]:
! pip install streamlit transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 9.6 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size